In [ ]:
import fastkaggle
import polars as pl

from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.linear_model import PoissonRegressor
from sklearn.model_selection import cross_validate, KFold
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer

In [ ]:
comp = "playground-series-s4e4"
path = fastkaggle.setup_comp(comp)

train = pl.read_csv(path/"train.csv").drop("id").with_columns(pl.col(pl.Float64).cast(pl.Float32))
test = pl.read_csv(path/"test.csv").drop("id").with_columns(pl.col(pl.Float64).cast(pl.Float32))
subm = pl.read_csv(path/"sample_submission.csv") 

In [ ]:
train.columns

In [ ]:
train.estimated_size(unit='mb')

In [ ]:
X= train.drop("Rings")
y=train.select(pl.last()).to_numpy().squeeze()

pipe = make_column_transformer(
    (OneHotEncoder(sparse_output=False), ["Sex"]),
    remainder="passthrough", verbose_feature_names_out=False
).set_output(transform="polars")

cv = KFold()
model = HistGradientBoostingRegressor()
mod_pipe = make_pipeline(pipe, model)
cross_validate(mod_pipe,X,y,cv=cv,scoring="neg_root_mean_squared_log_error")

In [ ]:
model = PoissonRegressor()
mod_pipe = make_pipeline(pipe, model)
cross_validate(mod_pipe,X,y,cv=cv,scoring="neg_root_mean_squared_log_error")

In [ ]:
model = HistGradientBoostingRegressor()
mod_pipe = make_pipeline(pipe, model)
mod_pipe.fit(X,y)
preds = mod_pipe.predict(test)

In [ ]:
ss = subm.with_columns("id",Rings=preds)
ss.write_csv(path/"subm.csv")

In [ ]:
from kaggle import api
api.competition_submit(path/"subm.csv", "routince boost tree", comp)

In [ ]:
train.drop("Rings").plot.kde()